In [1]:
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense,Input, Embedding
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../input/112k-korean-to-vietnamese-dataset/kr-vn-112k.csv',encoding='utf-8')
data.head()

,kr,vn
0,나는 매일 저녁 배트를 만나러 다락방으로 가요,tối nào tôi cũng đến gác mái để gặp bat
1,선생님 이문장이 이해가 안 가요,tôi không hiểu câu này của thầy
2,컴퓨터를 시작하면 시간이 너무 빠르게 가요,thời gian trôi qua quá nhanh khi tôi bắt đầu s...
3,나는 오늘 자정에 한국으로 돌아 가요,nửa đêm hôm nay tôi sẽ trở về hàn quốc
4,지금 잠을 자면 깨어나지 못할 거 같아서 지금 가요,bây giờ mà ngủ chắc không thức dậy nổi đâu nên...


In [3]:
print("Total data: ",data.shape[0])

Total data:  112856


In [4]:
# checking null values
data.isna().sum()

kr    0
vn    0
dtype: int64

In [5]:
# checking duplicated data
isDuplicated = data.duplicated().any()
if isDuplicated:
    total_duplicates = data.duplicated().sum()
    print("Total duplicate rows are: ",total_duplicates)
    data.drop_duplicates(inplace = True)

Total duplicate rows are:  13186


In [6]:
## sampling 90000 rows randomly
data = data.sample(n = 90000, random_state = 31)
data.shape

(90000, 2)

In [7]:
## changing uppercase to lowercase
data['kr'] = data['kr'].apply(lambda x: x.lower())
data['vn']=data['vn'].apply(lambda x: x.lower())

# Remove quotes
data['kr']=data['kr'].apply(lambda x: re.sub("'", '', x))
data['vn']=data['vn'].apply(lambda x: re.sub("'", '', x))

In [8]:
to_exclude = set(string.punctuation) # Set of all special characters
print("punctuations to exclude:: ",to_exclude)
# Remove all the special characters
data['kr']=data['kr'].apply(lambda x: ''.join(ch for ch in x if ch not in to_exclude))
data['vn']=data['vn'].apply(lambda x: ''.join(ch for ch in x if ch not in to_exclude))

punctuations to exclude::  {'}', '(', '@', '-', '&', '<', '=', '/', '+', ']', '"', '#', '$', '[', ')', ',', '{', '^', '\\', '`', '%', ';', '_', '.', '?', '>', '~', '*', '!', ':', '|', "'"}


In [9]:
from string import digits
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)

data['kr']=data['kr'].apply(lambda x: x.translate(remove_digits))
data['vn']=data['vn'].apply(lambda x: x.translate(remove_digits))

data['kr'] = data['kr'].apply(lambda x: re.sub("[영일이삼사오육칠팔구십]", "", x))

# Remove extra spaces
data['kr']=data['kr'].apply(lambda x: x.strip())
data['vn']=data['vn'].apply(lambda x: x.strip())
data['kr']=data['kr'].apply(lambda x: re.sub(" +", " ", x))
data['vn']=data['vn'].apply(lambda x: re.sub(" +", " ", x))

In [10]:
## adding start and end token to the target sentence
data['vn'] = data['vn'].apply(lambda x: "START_ " + x + " _END")

In [11]:
## counting length of english and hindi sentence
data['kr_length'] = data['kr'].apply(lambda x: len(x.split(' ')))
data['vn_length'] = data['vn'].apply(lambda x: len(x.split(' ')))

data.head()

,kr,vn,kr_length,vn_length
59284,결과는 우리가 해결해야 할 두 가지를 알려주고 있어요,START_ kết quả này cho thấy hai điều chúng ta ...,8,14
16694,세탁기를 다시 쓸 수 있으면 좋을 거에요,START_ giá mà có thể sử dụng lại máy giặt thì ...,7,15
73677,무용품나 가 중 가장 먼저 바꿔야 할 것은 뭐라고 생각하나요,START_ bạn nghĩ cái nào là vật dụng văn phòng ...,10,20
70879,그녀가 진을 보고 힘 난다고 했어요,START_ cô ấy nói rằng cô ấy nhìn thấy bức ảnh ...,6,18
77177,아쉽군요 줄다리기가 상금 제 많은 종목인데,START_ tiếc thật kéo co là hạng mục có nhiều t...,6,14


In [12]:
print("Maximum length of Korean Sentence: ", max(data['kr_length']))
print("Maximum length of Vietnam Sentence: ",max(data['vn_length']))

Maximum length of Korean Sentence:  22
Maximum length of Vietnam Sentence:  47


In [13]:
### Get English and Hindi Vocabulary
all_kr_words=set()
for kor in data['kr']:
    for word in kor.split():
        if word not in all_kr_words:
            all_kr_words.add(word)

all_vn_words=set()
for viet in data['vn']:
    for word in viet.split():
        if word not in all_vn_words:
            all_vn_words.add(word)
            

print("toral Korean words: ",len(all_kr_words))
print('total Vietnam words: ',len(all_vn_words))

toral Korean words:  92864
total Vietnam words:  8420


In [14]:
## using only sentence with length less than 20
mask1 = data['kr_length'] < 21
mask2 = data['vn_length'] < 21
data = data[mask1 & mask2]
data.shape

(85633, 4)

In [15]:
print("maximum length of Vietnam Sentence ",max(data['vn_length']))
print("maximum length of Korean Sentence ",max(data['kr_length']))

maximum length of Vietnam Sentence  20
maximum length of Korean Sentence  16


In [16]:
input_words = sorted(list(all_kr_words))
target_words = sorted(list(all_vn_words))
num_encoder_tokens = len(all_kr_words)
num_decoder_tokens = len(all_vn_words)

num_encoder_tokens, num_decoder_tokens

(92864, 8420)

In [17]:
num_decoder_tokens += 1 #for zero padding

In [18]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
print("Token for accelerating is: ",input_token_index['cfo로'])

Token for accelerating is:  50


In [19]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
print("Character for toker 50 is: ",reverse_input_char_index[50])

Character for toker 50 is:  cfo로


In [20]:
# splitting data
X_, y_ = data['kr'], data['vn']
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size = 0.2,random_state=42)
print("Total number of training data: ",X_train.shape[0])
print("Toral number of testing data: ",X_test.shape[0])

Total number of training data:  68506
Toral number of testing data:  17127


In [21]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

2022-05-15 07:31:32.720668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 07:31:32.826127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 07:31:32.826936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 07:31:32.828226: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    27859200    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    2526300     input_2[0][0]                    
______________________________________________________________________________________________

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])

In [23]:
max_length_src = 20
max_length_tar = 20 

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [24]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [25]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

2022-05-15 07:31:37.680303: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-05-15 07:31:44.307412: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


535/535 [==============================] - 107s 185ms/step - loss: 3.4098 - accuracy: 0.1557 - val_loss: 2.9617 - val_accuracy: 0.2303
Epoch 2/100
535/535 [==============================] - 98s 182ms/step - loss: 2.7124 - accuracy: 0.2851 - val_loss: 2.5510 - val_accuracy: 0.3189
Epoch 3/100
535/535 [==============================] - 98s 184ms/step - loss: 2.3797 - accuracy: 0.3536 - val_loss: 2.3331 - val_accuracy: 0.3653
Epoch 4/100
535/535 [==============================] - 98s 184ms/step - loss: 2.1567 - accuracy: 0.3983 - val_loss: 2.2071 - val_accuracy: 0.3918
Epoch 5/100
535/535 [==============================] - 98s 184ms/step - loss: 1.9823 - accuracy: 0.4341 - val_loss: 2.1295 - val_accuracy: 0.4067
Epoch 6/100
535/535 [==============================] - 99s 184ms/step - loss: 1.8353 - accuracy: 0.4655 - val_loss: 2.0914 - val_accuracy: 0.4163
Epoch 7/100
535/535 [==============================] - 99s 186ms/step - loss: 1.7061 - accuracy: 0.4946 - val_loss: 2.0809 - val_accura

In [26]:
model.save('./model.h5')

In [27]:
# Encode the input sequence to get the "thought vectors"
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [28]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k = -1

In [29]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Korean sentence:', X_train[k:k+1].values[0])
print('Actual Vietnam Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Vietnam Translation:', decoded_sentence[:-4])

Input Korean sentence: 그는 버디 퍼팅을 했지만 컵을 벗어났어요
Actual Vietnam Translation:  anh ấy đã đánh birdie putting nhưng anh ấy đã rời khỏi cốc 
Predicted Vietnam Translation:  anh ấy đã đánh hơn anh ấy đã nói ngoài vào mùa 


In [30]:
k = k + 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Korean sentence:', X_train[k:k+1].values[0])
print('Actual Vietnam Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Vietnam Translation:', decoded_sentence[:-4])

Input Korean sentence: 항상 좋은 인연으로 기억되길 바래요
Actual Vietnam Translation:  tôi hy vọng bạn sẽ luôn nhớ đến mối nhân duyên tốt đẹp 
Predicted Vietnam Translation:  tôi hy vọng bạn sẽ luôn nhớ đến mối nhân duyên


In [31]:
k = k + 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Korean sentence:', X_train[k:k+1].values[0])
print('Actual Vietnam Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Vietnam Translation:', decoded_sentence[:-4])

Input Korean sentence: 바로 공항에 전화해서 비행기 번호와 시간을 말하고 물어봐
Actual Vietnam Translation:  chính là sân bay gọi điện đến số hiệu máy bay và thời gian nói và hỏi đi 
Predicted Vietnam Translation:  chính là sân bay gọi điện đến số thời gian và nói ch


In [32]:
k = k + 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Korean sentence:', X_train[k:k+1].values[0])
print('Actual Vietnam Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Vietnam Translation:', decoded_sentence[:-4])

Input Korean sentence: 최종 목표는 실리콘밸리로 진출하고 세계 시장에 진입하는 것입니다
Actual Vietnam Translation:  mục tiêu cuối cùng là tiến vào thung lũng silicon và tiến vào thị trường thế giới 
Predicted Vietnam Translation:  mục tiêu cuối cùng là tiến vào thung lũng silico
